In [1]:
import numpy as np
import pandas as pd
#dataset import
df = pd.read_csv("../Data/data_8_nodes_yen.csv") #You need to change #directory accordingly
df.head()

,"X(1, 2)","X(1, 3)","X(2, 4)","X(2, 5)","X(3, 4)","X(3, 7)","X(4, 6)","X(5, 6)","X(5, 8)","X(6, 7)",...,"Y4(1, 3)","Y4(2, 4)","Y4(2, 5)","Y4(3, 4)","Y4(3, 7)","Y4(4, 6)","Y4(5, 6)","Y4(5, 8)","Y4(6, 7)","Y4(7, 8)"
0,10,40,30,50,30,20,20,10,90,70,...,0,1,0,0,0,1,0,0,1,1
1,90,50,50,30,30,80,80,30,30,80,...,1,0,0,1,0,1,1,1,0,0
2,10,60,80,50,40,50,30,90,80,30,...,1,0,0,1,0,1,0,0,1,1
3,80,20,40,60,50,80,90,60,80,80,...,1,0,0,1,0,1,1,1,0,0
4,20,70,50,40,20,70,80,80,10,10,...,0,1,0,0,0,1,1,1,0,0


In [2]:
edges = [[1, 2], [1, 3], [2, 4], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 8], [6, 7], [7, 8]]

In [3]:
number_edges = 11
number_steps = 4
start_node = 1
end_node = 8

In [4]:
X = df.iloc[:,:number_edges].values
y = df.iloc[:,number_edges:].values

In [5]:
X.shape

(100000, 11)

In [6]:
y.shape

(100000, 44)

In [7]:
from sklearn.model_selection import train_test_split
X1,X2,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [8]:
X2[1]

array([40, 40, 90, 30, 40, 30, 50, 30, 30, 70, 20], dtype=int64)

In [9]:
y_test[1]

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
      dtype=int64)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X1)
X_test= sc.fit_transform(X2)

In [11]:
X_train = np.repeat(X_train[:, np.newaxis, : ], number_steps, axis=1)
X_train.shape

(90000, 4, 11)

In [12]:
X_test = np.repeat(X_test[:, np.newaxis, : ], number_steps, axis=1)
X_test.shape

(10000, 4, 11)

In [13]:
y_train[0]

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1],
      dtype=int64)

In [14]:
y_train_2 = np.reshape(y_train,(y_train.shape[0], number_steps, number_edges))
y_train_2.shape

(90000, 4, 11)

In [15]:
y_train_2[0]

array([[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1]], dtype=int64)

In [16]:
y_test_2 = np.reshape(y_test,(y_test.shape[0], number_steps, number_edges))
y_test_2.shape

(10000, 4, 11)

In [17]:
X2[1,:]

array([40, 40, 90, 30, 40, 30, 50, 30, 30, 70, 20], dtype=int64)

In [18]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,TimeDistributed,GRU
# Neural network
model = Sequential()
model.add(GRU(64, input_shape=(number_steps, number_edges), 
                    return_sequences=True))
model.add(TimeDistributed(Dense(number_edges, activation='sigmoid')))

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 4, 64)             14784     
                                                                 
 time_distributed (TimeDistr  (None, 4, 11)            715       
 ibuted)                                                         
                                                                 
Total params: 15,499
Trainable params: 15,499
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(loss='BinaryCrossentropy', optimizer='adam')

In [21]:
model.fit(X_train, y_train_2, epochs=100, batch_size=64)

Epoch 1/100
1407/1407 [==============================] - 7s 4ms/step - loss: 0.3740
Epoch 2/100
1407/1407 [==============================] - 6s 4ms/step - loss: 0.2287
Epoch 3/100
1407/1407 [==============================] - 7s 5ms/step - loss: 0.1832
Epoch 4/100
1407/1407 [==============================] - 6s 5ms/step - loss: 0.1583
Epoch 5/100
1407/1407 [==============================] - 5s 4ms/step - loss: 0.1413
Epoch 6/100
1407/1407 [==============================] - 6s 4ms/step - loss: 0.1292
Epoch 7/100
1407/1407 [==============================] - 5s 4ms/step - loss: 0.1196
Epoch 8/100
1407/1407 [==============================] - 5s 4ms/step - loss: 0.1119
Epoch 9/100
1407/1407 [==============================] - 6s 4ms/step - loss: 0.1055
Epoch 10/100
1407/1407 [==============================] - 6s 4ms/step - loss: 0.1001
Epoch 11/100
1407/1407 [==============================] - 7s 5ms/step - loss: 0.0956
Epoch 12/100
1407/1407 [==============================] - 6s 4ms/step - lo

In [22]:
len(model.layers)

2

In [23]:
pred=model.predict(X_train)

2813/2813 [==============================] - 6s 2ms/step


In [24]:
pred.shape

(90000, 4, 11)

In [25]:
pred[0]

array([[5.76682532e-06, 9.99994218e-01, 3.84337181e-04, 2.69480893e-06,
        1.43590872e-03, 9.99981403e-01, 2.07012163e-05, 1.28822985e-05,
        6.59681100e-05, 3.82518338e-05, 9.99934077e-01],
       [9.96796370e-01, 3.20299668e-03, 9.78069067e-01, 6.21381775e-03,
        6.08977973e-01, 9.91390586e-01, 2.60241271e-04, 5.65926748e-05,
        5.35128592e-03, 7.34674511e-03, 9.94651973e-01],
       [9.99999881e-01, 5.56974769e-08, 2.00018520e-03, 9.99990463e-01,
        1.29641718e-04, 1.31735011e-04, 1.52620196e-05, 3.58479014e-07,
        9.99982893e-01, 4.40362373e-06, 1.71264273e-05],
       [1.00000000e+00, 1.19106778e-08, 9.99999821e-01, 4.15192062e-06,
        9.95739356e-06, 2.63780876e-05, 9.99997735e-01, 9.18250080e-05,
        1.04360006e-04, 9.99680638e-01, 9.99895692e-01]], dtype=float32)

In [26]:
y_train_2[0]

array([[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1]], dtype=int64)

In [27]:
import copy
def cost_path(x,y):
    cost = 0
    path = []
    for i in range(number_edges):
        if y[i]==1:
            path.append(edges[i])
            cost+= x[i]
    if(len(path)!=0):
        path_2 = copy.deepcopy(path)
        path_nodes = [start_node]
        if(path[0][0]!=start_node):
            cost = 0
        else:
            path_nodes.append(path[0][1])
            del path_2[0]
            condition = True
            while(len(path_2)!=0 and condition):
                condition = False
                for i in range(len(path_2)):
                    if(path_nodes[-1] in path_2[i]):
                        path_2[i].remove(path_nodes[-1])
                        path_nodes.append(path_2[i][0])
                        path_2.pop(i)
                        condition = True
                        break
        if(path_nodes[-1]!=end_node or len(path_2)!=0):
            cost = 0
    else:
        cost = 0
    return cost

In [28]:
cost_path(X1[1],y_train[1])

120

In [29]:
accuracy = np.zeros(number_steps+1)
for i in range(pred.shape[0]):
    condition_1 = True
    for t in range(number_steps):
        output = copy.deepcopy(pred[i][t])
        condition_2 = True
        for j in range(len(output)):
            output[j] = round(output[j])
        if(cost_path(X1[i],output) != cost_path(X1[i],y_train_2[i][t])):
            condition_2 = False
            condition_1 = False
        if condition_2:
            accuracy[t] += 1
    if condition_1:
        accuracy[number_steps] += 1
accuracy = np.round(accuracy / pred.shape[0] * 100,2)
accuracy

array([95.75, 94.67, 94.83, 92.74, 84.05])

In [30]:
pred_test=model.predict(X_test)

313/313 [==============================] - 1s 2ms/step


In [31]:
accuracy_test = np.zeros(number_steps+1)
for i in range(pred_test.shape[0]):
    condition_1 = True
    for t in range(number_steps):
        output = copy.deepcopy(pred_test[i][t])
        condition_2 = True
        for j in range(len(output)):
            output[j] = round(output[j])
        if(cost_path(X1[i],output) != cost_path(X1[i],y_test_2[i][t])):
            condition_2 = False
            condition_1 = False
        if condition_2:
            accuracy_test[t] += 1
    if condition_1:
        accuracy_test[number_steps] += 1
accuracy_test = np.round(accuracy_test / pred_test.shape[0] * 100,2)
accuracy_test

array([95.47, 94.25, 93.26, 89.43, 80.94])

In [34]:
model.summary()
print(accuracy)
print(accuracy_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 4, 64)             14784     
                                                                 
 time_distributed (TimeDistr  (None, 4, 11)            715       
 ibuted)                                                         
                                                                 
Total params: 15,499
Trainable params: 15,499
Non-trainable params: 0
_________________________________________________________________
[95.75 94.67 94.83 92.74 84.05]
[95.47 94.25 93.26 89.43 80.94]
